# *Flexibility and Sustainability LNCMI*

## Informations générales

**Background of the study**: This notebook presents a study of energy flexibility and waste heat recovery levers applied to an electricity-intensive research facility, the National Laboratory for Intense Magnetic Fields (LNCMI). This analysis was carried out using the OMEGAlpes modeling tool. The objective is to optimize the six-monthly schedule of experiments and the operation of a waste heat recovery system with the aim of reducing the site's greenhouse gas emissions. A systemic approach is adopted to assess CO2 emissions, in order to integrate phenomena such as rebound effects or the transfer of undesirable impacts.


**Licence**: [Apache 2.0](https://www.apache.org/licenses/LICENSE-2.0.html)  
mybinder link for online use [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/git/https%3A%2F%2Fgricad-gitlab.univ-grenoble-alpes.fr%2Fomegalpes%2Fomegalpes_examples/HEAD?labpath=%2Fnotebooks%2FPhD_2022_Sacha_Hodencq%2FLNCMI_planning.ipynb)

**Reference**:  Associated article  : Séverin Valla, Nana Twum-Duah, Sacha Hodencq, Francois Debray, Frédéric Wurtz. Flexibilité pour la durabilité des systèmes multiénergie : le cas d’une grande infrastructure de recherche. Symposium de Génie Électrique SGE 2025, Jul 2025, Toulouse, France. hal-05235968￿"  

**Outil** : [OMEGAlpes](https://gricad-gitlab.univ-grenoble-alpes.fr/omegalpes/omegalpes), version 0.4.1, Apache 2.0 licence  

**Liens vers d'autres études de cas associées**:
   - See in particular the ECOS conference article "[Flexible waste heat management and recovery for an electro-intensive industrial process through energy/exergy criteria](https://hal.archives-ouvertes.fr/hal-03290126)" presents the same case study.   
   - section IV.2.6.1 de la [PhD. Theses of Sacha Hodencq](http://www.theses.fr/s225905).
   - E le [notebook LNCMI](https://gricad-gitlab.univ-grenoble-alpes.fr/omegalpes/omegalpes_examples/-/blob/master/notebooks/LNCMI.ipynb) à des fins de médiation.
   - This article also uses the method [ORUCE](https://hal.archives-ouvertes.fr/hal-03341883) (Open and Reproducible Use Case for Energy).  

**Développeur – institution** : Nana Kofi (nana-kofi-baabu.twum-duah@g2elab.grenoble-inp.fr) - G2Elab, Séverin Valla (severin.valla@grenoble-inp.fr) - G2ELab

## Table des matières

- [Informations générales](#Informations-générales)
- [Description des données](#Spécifications-des-données)
- [Présentation des modèles énergetiques](#Présentation-des-modèles-énergetiques)
  - Contexte générale
  - Scénarios etudiés
  - Modèles utilisés
- [Processus de modélisation énergétique](#Processus-de-modélisation-énergétique)
  - Scénario 1 – REF
  - Scénario 2 – REF_HEBDO
  - Scénario 3 – VALO_HEBDO
- [Résultats](#Résultats)

**Package requirement specifications**  
  
Also see [OMEGAlpes documentation](https://omegalpes-examples.readthedocs.io/en/latest/jupyter.html) for using the relevant environement depending on your situation.  
Code to execute if you are using mybinder (remove the "#" in front of the code lines) :

In [1]:
#############################################################################
#import os
#os.system('pip install -r NB_requirements/LNCMI_planning_requirements.txt')
#############################################################################

In [2]:
# Import package

import pulp
import os
import math
import plotly.express as px
import plotly.subplots as sp

from IPython.display import clear_output

import pandas as pd
from pulp import LpStatus, GUROBI_CMD

import numpy as np

import matplotlib.lines as lines
from pandas import Series
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# -----------------------------------------------------------------------------

from omegalpes.energy.energy_nodes import EnergyNode
from omegalpes.energy.units.consumption_units import FixedConsumptionUnit, VariableConsumptionUnit, ShiftableConsumptionUnit
from omegalpes.energy.units.conversion_units import  HeatPump, ConversionUnit
from omegalpes.energy.units.production_units import ProductionUnit, VariableProductionUnit
from omegalpes.energy.units.storage_units import StorageUnit
from omegalpes.general.optimisation.model import OptimisationModel
from omegalpes.general.time import TimeUnit
from omegalpes.general.utils.plots import plt, plot_quantity_bar, \
    plot_node_energetic_flows

from omegalpes.energy.energy_types import elec, thermal
from omegalpes.general.optimisation.elements import DefinitionDynamicConstraint, TechnicalConstraint, ActorConstraint, DefinitionConstraint

DEPRECATED: please now use SingleConversionUnit with relevant energytypes


## Description des données

### Consommation électrique du LNCMI

The ``conso_lncmi_v7`` dataset contains the LNCMI laboratory's electricity consumption data for one year. It is a hypothetical reconstruction of a typical year based on representative segments from previous years, eliminating anomalies caused by exceptional events. The data for these segments was obtained from RTE. It corresponds to the instantaneous power in kW, taken from the RTE transmission network at hourly intervals.

|#|Key |Parameter |
|---|---|---|
| 1 | name |cons_lncmi_v7.csv| 
| 2 | title | LNCMI electricity consumption data for a typical year| 
| 3 | id | |
| 4 | description |Instantaneous power in kW, taken from the RTE network, with a time step of one hour |
| 5 | language | |
| 6 | subject |  |
| 7 | keywords | |
| 8 | publicationDate | |
| 9 | context | |
| 10 | spatial |Grenoble, 21 avenue des Martyrs| 
| 11 | temporal |Hourly time step|
| 12 | source | RTE|
| 13 | licenses | |
| 14 | contributors |  |
| 15 | resources | |

- Loading

In [3]:
# Définition des paramètres temporels
DT=1
START_DATE = "01/01/2018 00:00"
END_DATE = "30/12/2018 23:00"  # Était "31/12/2018 23:00" mais on reduit d'un jour pour coller à 52 semaines dans l'année
start = pd.to_datetime(START_DATE, dayfirst=True)
end = pd.to_datetime(END_DATE, dayfirst=True)

# Consommation électrique
indus_cons_file = pd.read_csv('data/cons_lncmi_v7.csv', sep=';', index_col='date', parse_dates=True, dayfirst=True)
indus_cons_file = indus_cons_file.loc[start:end]
INDUS_CONS = indus_cons_file['P [kW]'].values.tolist()

# Creation du dataframe df et ajout des consommations électriques 
df = pd.DataFrame(index=indus_cons_file.index)
df["Electricity_Consumption[kW]"] = INDUS_CONS
df["Electricity_Consumption[MW]"] = df["Electricity_Consumption[kW]"] / 1000

### CNRS heat consumption

The load_cnrs dataset contains heat consumption data for the CNRS site in Grenoble over one year. It is a fictitious reconstruction of a typical year, based on representative segments from 2020 and 2021. The data for these segments was obtained from the STL (Technical and Logistics Service) of the CNRS site in Grenoble. It corresponds to the instantaneous thermal power in kW, taken from the CCIAG urban heating network, with a time step of 10 minutes.

|#|Key |Parameter |
|---|---|---|
| 1 | name | load_cnrs.csv|
| 2 | title |CNRS heat consumption data for a typical year| 
| 3 | id | |
| 4 | description |Instantaneous thermal power in kW, taken from the CCIAG network, with a time step of 10 minutes |
| 5 | language | |
| 6 | subject |  |
| 7 | keywords | |
| 8 | publicationDate | |
| 9 | context | |
| 10 | spatial | Grenoble, 21 avenue des Martyrs | 
| 11 | temporal | 10-minute time step|
| 12 | source |STL|
| 13 | licenses |  |
| 14 | contributors | |
| 15 | resources | |

- Loading

In [4]:
# Chaleur
heat_load_file = pd.read_csv('data/load_cnrs.csv', sep=';')
heat_load_file['Temps'] = pd.to_datetime(heat_load_file['Temps'], format='%d/%m/%Y %H:%M')
heat_load_file.set_index('Temps', inplace=True)
heat_load_file = heat_load_file.resample('H').mean()
heat_load_file = heat_load_file.loc[start:end]
HEAT_LOAD = heat_load_file['Puiss_CNRS (KW)'].values.tolist()

# Ajout au dataframe df
df["Heat_demand[kW]"] = HEAT_LOAD
df["Heat_demand[MW]"]  = df["Heat_demand[kW]"] /1000

C:\Users\twumduan\AppData\Local\Temp\ipykernel_39928\3711212186.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  heat_load_file = heat_load_file.resample('H').mean()


In [5]:
print(df.columns)

# 2. Vérifier le pas de temps (différence moyenne entre 2 index successifs)
print(df.index.to_series().diff().value_counts())

# Ou uniquement le pas le plus fréquent :
print(df.index.to_series().diff().mode())

# 3. Obtenir la valeur maximale de la demande de chaleur (en MW)
print(df["Heat_demand[MW]"].max())

Index(['Electricity_Consumption[kW]', 'Electricity_Consumption[MW]',
       'Heat_demand[kW]', 'Heat_demand[MW]'],
      dtype='object')
date
0 days 01:00:00    8735
Name: count, dtype: int64
0   0 days 01:00:00
Name: date, dtype: timedelta64[ns]
1.9610783333333337


### Carbon intensity of electricity and heat 

The carbon intensity data for French electricity in 2023 was retrieved from the Electricity Map platform and compiled in the all_data_2023 dataset. It corresponds to the amount of CO2 emitted per kilowatt-hour of electricity (in gCO2/kWh), on an hourly basis.
For the carbon intensity of heat drawn from the CCIAG heat network, a fictitious estimate was made based on an understanding of the network, as the actual data is confidential.

|#|Key |Parameter |
|---|---|---|
| 1 | name | all_data_2023.csv|
| 2 | title |Carbon intensity of electricity in France during 2023| 
| 3 | id | |
| 4 | description | amount of CO2 emitted per kilowatt-hour of electricity consumed (in gC02/kWh), on an hourly basis |
| 5 | language | |
| 6 | subject |  |
| 7 | keywords | |
| 8 | publicationDate | |
| 9 | context | |
| 10 | spatial | France | 
| 11 | temporal | 1-hour time step|
| 12 | source |Electricity Map|
| 13 | licenses |  |
| 14 | contributors | |
| 15 | resources | |

- Loading

In [6]:
# CO2 elec
df_2023 = pd.read_csv("Data/all_data_2023.csv", sep=",", decimal=".", parse_dates=True, index_col=0)
df_2023_resampled = df_2023['Taux de Co2'].resample('H').mean()
df_2023_resampled = df_2023_resampled.iloc[:-24]  # Retirer les 24 dernières heures

# Ajout au dataframe df
assert len(df_2023_resampled) == len(df), "Les longueurs ne correspondent pas"
df["Taux de Co2"] = df_2023_resampled.values

C:\Users\twumduan\AppData\Local\Temp\ipykernel_39928\3634733360.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_2023_resampled = df_2023['Taux de Co2'].resample('H').mean()


As the actual data on the carbon intensity of the CCIAG's heat supply is confidential, a theoretical estimate has been made based on an understanding of the district heating network. This estimate corresponds to the average monthly amount of CO2 emitted per kilowatt-hour of heat (in gCO2/kWh).

In [7]:
# CO2 chaleur
cciag_co2 = { 1 : 100.78 ,           # en gC02/KWh
              2 : 81.28,
              3 : 52.02 , 
              4 : 52.02,
              5 : 52.02,
              6 : 48.77,
              7 : 3.25,
              8 : 0,
              9 : 74.78,
              10 : 45.52,
              11 : 61.77,
              12 : 87.78,
              }

# Ajout au dataframe df
df["cciag_co2"] = df.index.month.map(cciag_co2)


- Plotting

In [8]:
layout = {
    "font":{"size" : 13},
    
    "template": "plotly_white",
    "title" : "  Carbon Intensity for electric and heating networks[g CO2/kWh] - France",
    "width":1200,
    "height":500,

    'xaxis': {"showline": False,"linecolor": "black", "linewidth": 2,"tickangle" : -45,
            "title_text" : "Date","tickangle" : 45
             },

    "yaxis": {"showline": True,"linecolor": "black", "linewidth": 2,
              'zerolinecolor':'black',
               "side": 'left',
              "title_text" : "Grid Carbon Intensity [g CO2/kWh] ",
    },
    "legend" : {"tracegroupgap":8 ,"font_size": 14, "orientation":"h", "yanchor":"bottom",
    "y":-0.35,"x":0.120, "title" : ""}
}

fig = px.line(df[["cciag_co2","Taux de Co2"]] )
fig.data[1].name = "RTE"
fig.update_layout(layout)

fig

- Plotting carbon intensities and heat demand for the site: factors influencing optimization

In [9]:
import plotly.express as px
import plotly.graph_objects as go

# Lissage sur 12 heures pour l'intensité carbone
df_smooth = df.copy()
df_smooth["Taux de Co2"] = df["Taux de Co2"].rolling(window=12, center=True).mean()
df_smooth["cciag_co2"] = df["cciag_co2"].rolling(window=12, center=True).mean()

fig = px.line(df_smooth, y=["Taux de Co2","cciag_co2"],
              labels={"value": "Carbon Intensity [gCO₂/kWh]", "variable": "Source"},
              title="French Electric Grid Carbon Intensity vs CNRS Grenoble Campus Heat Demand")

fig.data[0].name = "RTE (Electric)"
fig.data[1].name = "CCIAG (Heat)"

# Ajout de la demande de chaleur brute (sans modification) en gris
fig.add_trace(go.Scatter(
    x=df_smooth.index,
    y=df_smooth["Heat_demand[MW]"],  # Valeurs brutes, non modifiées
    name="CNRS Heat Demand",
    yaxis="y2",
    line=dict(color="gray", width=2)
))

# Mise à jour du layout
fig.update_layout(
    template="plotly_white",
    # width=100,
    height=700,
    font=dict(size=13),
    xaxis=dict(
        title="Date",
        showline=False,
        linecolor="black",
        # linewidth=2,
        tickangle=45
    ),
    yaxis=dict(
        title="Carbon Intensity [gCO₂/kWh]",
        linecolor="black",
        linewidth=2
    ),
    yaxis2=dict(
    title={
        "text":"Heat Demand-CNRS [MW]",
        "font":{"color" : "gray"}
          },
    overlaying="y",
    side="right",
    showgrid=False,
    linecolor="gray",
    tickfont=dict(color="gray"),
    range=[0, 5]
            )
    ,
    legend=dict(
        title = "Variable",
        orientation="h",
        yanchor="bottom",
        y=-0.35,
        x=0.35,
        font=dict(size=13)
    )
)

fig.show()

### Cost of electricity and heat (CCIAG)

Electricity and heating rates are based on LNCMI contracts with RTE and CCIAG. They vary according to four rate periods (summer/winter, peak/off-peak) for electricity and by month for heating.  

As this data is confidential, it is not published in this notebook. The results displayed here are based on these actual rates, but if you wish to run simulations again, you will need to use fictitious data (to be entered below).

In [10]:
# tarifs = pd.read_excel("data/couts_energies.xlsx", sheet_name="elec")

# # --- Initialisation du coût ---

# df["cost_MWh"] = 0.0

# # --- Attribution des coûts selon les conditions temporelles ---
# # hiver = nov à mars, été = avr à oct ; jour = 6h à 22h, nuit = 22h à 6h

# df.loc[((df.index.month <= 3) | (df.index.month >= 11)) & ((df.index.hour >= 22) | (df.index.hour < 6)), "cost_MWh"] = tarifs.loc[tarifs["description"] == "hiver_nuit", "cost_MWh"].iloc[0]

# df.loc[((df.index.month <= 3) | (df.index.month >= 11)) & ((df.index.hour < 22) & (df.index.hour >= 6)), "cost_MWh"] = tarifs.loc[tarifs["description"] == "hiver_jour", "cost_MWh"].iloc[0]

# df.loc[(df.index.month > 3) & (df.index.month < 11) & ((df.index.hour >= 22) | (df.index.hour < 6)), "cost_MWh"] = tarifs.loc[tarifs["description"] == "ete_nuit", "cost_MWh"].iloc[0]

# df.loc[(df.index.month > 3) & (df.index.month < 11) & ((df.index.hour < 22) & (df.index.hour >= 6)), "cost_MWh"] =  tarifs.loc[tarifs["description"] == "ete_jour", "cost_MWh"].iloc[0]

# df["cost_kwh"] = df["cost_MWh"]/1000

# ============================
# pour run cette partie du notebook entrer des valeurs pour les differentes periodes 

df["cost_MWh"] = 0
df.loc[((df.index.month <= 3) | (df.index.month >= 11)) & (( df.index.hour >= 22) | ( df.index.hour < 6)),"cost_MWh"] = 70
df.loc[((df.index.month <= 3) | (df.index.month >= 11)) & (( df.index.hour < 22) & ( df.index.hour >= 6)),"cost_MWh"] = 110

df.loc[(df.index.month > 3) & (df.index.month < 11) & (( df.index.hour >= 22) | ( df.index.hour < 6)),"cost_MWh"] = 45
df.loc[(df.index.month > 3) & (df.index.month < 11) & ( df.index.hour < 22) & ( df.index.hour >= 6),"cost_MWh"] = 55
df["cost_kwh"] = df["cost_MWh"]/1000

# ============================

In [11]:
# heat_cost_df = pd.read_excel("Data/couts_energies.xlsx", sheet_name="chaleur")

# # Conversion en dictionnaire {mois: coût}
# cciag_cost = dict(zip(heat_cost_df["month"], heat_cost_df["cciag_cost_MWh"]))

# # Ajout au dataframe principal
# df["cciag_cost"] = df.index.month.map(cciag_cost)

# ============================
# pour run cette partie du notebook entrer des valeurs pour les differents mois  

cciag_cost = {                       # en euro/MWh
   1: 80,
   2: 80,
   3: 100,
   4: 45,
   5: 55,
   6: 55,
   7: 55,
   8: 55,
   9: 55,
   10: 80,
   11: 90,
   12: 92,
}
df["cciag_cost"] = df.index.month.map(cciag_cost)

# ============================

## Energy Models

### General context

The system under study comprises the LNCMI experimental facilities and the heating network of the CNRS site in Grenoble, where the LNCMI is located.

The LNCMI facilities include:  
- Direct current electromagnets, consuming approximately 10 GWh/year, with a power output of up to 30 MW. All the electricity is converted into heat by the Joule effect.
- A cooling system that dissipates this heat into the Isère River.

For its part, the CNRS heating network supplies buildings (heating load) between October and April, for a total of approximately 2 GWh/year. This heat is supplied by the CCIAG urban network production unit.

Below are diagrams of the subsystems studied.

<div style="display: flex; justify-content: space-around; align-items: flex-start; gap: 20px;">

  <figure style="text-align: center; width: 48%;">
    <img src="figures/installations_LNCMI_v2.PNG" style="width:70%;">
    <figcaption><b>Figure 1 :</b> Diagram of the LNCMI facilities</figcaption>
  </figure>

  <figure style="text-align: center; width: 48%;">
    <img src="figures/reseau_chaleur_CNRS.PNG" style="width:60%;">
    <figcaption><b>Figure 2 :</b> Diagram of the Heating network of CNRS Grenoble</figcaption>
  </figure>

</div>

### Scenarios studied

In this notebook, three scenarios are analyzed to understand the impact of waste heat recovery (lost heat) and rescheduling of experimental campaigns:

- **REF**: reference scenario, without heat recovery or rescheduling. ;
- **REF_WEEKLY**: scenario with weekly replanning of experiments;
- **VALO_WEEKLY**:  scenario combining weekly replanning and waste heat recovery using a heat pump and thermal storage.  

These scenarios are presented in the article associated with the notebook.

### Modèles utilisés
  
La modélisation du système change selon le scénario étudié.

### Models used

The system modeling varies depending on the scenario studied.


**Scenario 2 – REF_WEEKLY**

This scenario examines the weekly rescheduling of experiments carried out at the LNCMI. The aim is to adapt the experimental schedule in order to reduce CO₂ emissions, taking into account the variability of the carbon intensity of electricity throughout the year.  

To do this, we use an OMEGAlpes model that allows us to reorganize the weeks of experimentation according to the carbon intensity of electricity. The model is composed of several interconnected elements:

- *Electricity supplier*: represents the national electricity grid, capable of powering the experiments. It is modeled as a variable production unit, whose power automatically adjusts according to demand.  
- *LNCMI experiments*: grouped into 52 weekly blocks, each corresponding to a week of experimental activity. Each block represents the laboratory's weekly electricity consumption and constitutes a consumption unit.
- *Electricity node*: acts as an energy connection point between the electricity supplier and all consumption units.  
  
Below is the OMEGAlpes diagram for scenario 2.  


<p align="center">
  <img src="figures/Diagrame_OMEGAlpes_scenario_2.PNG" style="width: 30%;">
</p>
<p style="text-align:center;"><b>Figure 3 :</b> OMEGAlpes diagram for scenario 2</p>  

The optimization objective is to minimize CO₂ emissions linked to electricity consumption over the year.

At the end of this optimization, a reorganized experimental schedule is obtained, and an annual power profile is reconstructed based on the new weekly organization.

Main constraints:  
- Each block has a fixed energy and equivalent power distributed over 168 hours;
- Each block can only be used once (binary `ublock`).





**Scenario 3 – VALO_HEBDO**

In scenario 3, we examine the combined levers of weekly replanning of LNCMI experiments and the integration of a waste heat recovery system.

This system recovers some of the heat produced by the Joule effect in the electromagnets and feeds it back into the CNRS heating network.
The system comprises:
- a heat exchanger
- thermal storage
- a heat pump
- pipes

The thermal storage system smooths out the highly variable heat production to better match the needs of the CNRS. The heat pump is used to raise the temperature of the water from the magnet cooling system (≈35°C) to a level suitable for the heating network (≈50°C). A diagram of the heat recovery system is shown below.


<p align="center">
  <img src="figures/systeme_recuperation_chaleur_fatale.PNG" style="width:70%;">
</p>
<p style="text-align:center;"><b>Figure 4 :</b> Diagram of the waste heat recovery system under study at the LNCMI</p>


To study the combination of these two levers, two successive optimizations are performed on two different OMEGAlpes models.

<u> 1. Replanning model  </u>

The first OMEGAlpes model is based on the REF_HEBDO scenario, but this time takes into account the potential for heat recovery.  
  
The system includes:  
- *Electricity supplier*: national grid, modeled as a variable production unit.  
- *LNCMI experiments*: 52 weekly blocks, representing electricity consumption and heat produced. Modeled as conversion units (electricity → heat).  
- *Heat rejection*: represents the heat dissipated into the Isère river, i.e., the unused portion.
- *CNRS consumption*: fixed consumption unit corresponding to annual heating demand.
- *CCIAG heat production*: urban network production, modeled as a variable production unit.  
- *Overconsumption*: takes into account the rebound effect due to the overconsumption of electricity by the magnets when heat is recovered and the consumption of the heat pump.  

Thermal storage is not modeled directly here (the time step being 168 hours), but its overall effect is taken into account in the assessment of recoverable heat.

Below is the OMEGAlpes diagram for model 1 of scenario 3.   

<p align="center">
  <img src="figures/Diagrame_OMEGAlpes_Scenario_3_Model_1.PNG" style="width:70%;">
</p>
<p style="text-align:center;"><b>Figure 5 :</b> OMEGAlpes diagram for model 1 of scenario 3</p>

The optimization objective is still to reduce GHG emissions, but this time it depends on:
- the carbon intensity of electricity and heat,
- the CNRS's heat demand,
- and the rebound effects associated with excessive electricity consumption (magnets and heat pumps).

<u> 2. Evaluation model  </u>

In a second step, a more detailed OMEGAlpes model is used to simulate the optimal management of thermal storage and heat recovery based on the replanned consumption profile.

The system includes:
- *Electricity supplier*: supplying the experiments and the heat pump.
- *LNCMI experiments*: rearranged annual profile of electricity consumption and heat production by the magnets. A single conversion unit.
- *Heat rejection*
- *Thermal storage*: represents a generic thermal storage system, defined as a storage unit
- *Heat pump*: represents a water/water heat pump. It is modeled as a conversion unit
- *CCIAG heat production unit*
- *CNRS heating consumption*

The overconsumption of magnets is not represented in this model. It is calculated post-modeling with temperature profiles obtained from the LNCMI and then added to the bills.

Below is the OMEGAlpes diagram for model 2 of scenario 3.


<p align="center">
  <img src="figures/Diagrame_OMEGAlpes_Scenario_3_Model_2.PNG" style="width:70%;">
</p>
<p style="text-align:center;"><b>Figure 6 :</b> OMEGAlpes diagram for model 2 of scenario 3</p>

## Mathematical Formulation of the MILP Optimization Model

### 1. Sets and Indices

* $t \in \mathcal{T}$ : discrete time periods (weekly resolution, $\Delta t = 168$ h)
* $j \in \mathcal{J}$ : electricity consumption blocks (indexed by week)

---

### 2. Parameters

#### Time and Energy

* $\Delta t$ : duration of one time step (h)
* $E_j$ : electrical energy associated with consumption block $j$ (kWh)
* $P^{\max}_{j}$ : maximum electrical power of block $j$ (kW)
* $\eta$ : electrical-to-thermal conversion efficiency (0.85)

#### Electricity Grid

* $c^{\text{grid}}_t$ : operating cost of electricity import (€/MWh)
* $\gamma^{\text{grid}}_t$ : CO$_2$ emission factor of electricity (kgCO$_2$/MWh)
* $P^{\min,grid}_t$, $P^{\max,grid}_t$ : minimum and maximum grid import power (kW)

#### District Heating (CCIAG)

* $\gamma^{\text{cciag}}_t$ : CO$_2$ emission factor of thermal production (kgCO$_2$/MWh)
* $P^{\max,cciag}$ : maximum thermal production capacity (kW)

#### Heat Demand

* $D^{\text{heat}}_t$ : fixed heat demand (kW)

#### Overconsumption Coefficient

* $\alpha = 0.045$ : proportional coefficient linking heat recovery to electrical overconsumption

---

### 3. Decision Variables

#### Continuous Variables

* $P^{\text{grid}}_t \ge 0$ : electrical power imported from the grid (kW)
* $P^{\text{cciag}}_t \ge 0$ : thermal power produced by CCIAG (kW)
* $P^{\text{over}}_t \ge 0$ : additional electrical overconsumption (kW)
* $P^{\text{rej}}_t \ge 0$ : rejected heat (kW)

For each block $j$:

* $P^{\text{elec}}_{j,t} \ge 0$ : electrical power of block $j$ (kW)
* $P^{\text{therm}}_{j,t} \ge 0$ : thermal power recovered from block $j$ (kW)

#### Binary Variables

* $u_{j,t} \in {0,1}$ : activation variable of block $j$ at time $t$

---

### 4. Objective Function

The optimization minimizes total CO$_2$ emissions:

$$
\min \sum_{t \in \mathcal{T}} \left(
\gamma^{\text{grid}}_t \cdot E^{\text{grid}}_t

* \gamma^{\text{cciag}}_t \cdot E^{\text{cciag}}_t
  \right)
  $$

with:

$$
E^{\text{grid}}_t = P^{\text{grid}}_t \Delta t
$$

$$
E^{\text{cciag}}_t = P^{\text{cciag}}_t \Delta t
$$

---

### 5. Constraints

#### 5.1 Electrical Power Balance

For each time step $t$:

$$
P^{\text{grid}}*t = \sum*{j \in \mathcal{J}} P^{\text{elec}}_{j,t} + P^{\text{over}}_t
$$

---

#### 5.2 Thermal Power Balance

At the heat node:

$$
\sum_{j \in \mathcal{J}} P^{\text{therm}}_{j,t} + P^{\text{cciag}}_t
= D^{\text{heat}}_t + P^{\text{rej}}_t
$$

---

#### 5.3 Conversion Constraints (Block Level)

For each block $j$ and time $t$:

$$
P^{\text{therm}}*{j,t} = \eta \cdot P^{\text{elec}}*{j,t}
$$

Power bounds:

$$
0 \le P^{\text{elec}}_{j,t} \le P^{\max}*j u*{j,t}
$$

Energy constraint (each block must deliver exactly its predefined energy):

$$
\sum_{t \in \mathcal{T}} P^{\text{elec}}_{j,t} \Delta t = E_j
$$

---

#### 5.4 Single Use Constraint (Per Block)

Each block can be activated at most once:

$$
\sum_{t \in \mathcal{T}} u_{j,t} \le 1
$$

---

#### 5.5 Mutual Exclusivity Constraint

At each time step, at most one block can operate:

$$
\sum_{j \in \mathcal{J}} u_{j,t} \le 1
$$

---

#### 5.6 Grid Operational Bounds

$$
P^{\min,grid}_t \le P^{\text{grid}}_t \le P^{\max,grid}_t
$$

---

#### 5.7 CCIAG Capacity Constraint

$$
0 \le P^{\text{cciag}}_t \le P^{\max,cciag}
$$

---

#### 5.8 Overconsumption Constraint

The electrical overconsumption induced by heat recovery is modeled as:

$$
P^{\text{over}}_t = \alpha \left(D^{\text{heat}}_t - P^{\text{cciag}}_t\right)
$$

This linear relationship links avoided district heating production to additional electrical consumption.

---





## Energy modelling process

### Scenario 1 : REF 

In [12]:
# ---Bilan LNCMI actuel  ---

print("\n--- BILAN DES CONSOMMATIONS, EMISSIONS ET COUTS du LNCMI actuel ---")

# Bilan : de la consommation totale de chaleur et d'électricité en MWh 

total_heat_demand_MWh = df["Heat_demand[MW]"].dropna().sum()
total_electricity_consumption_MWh = df["Electricity_Consumption[MW]"].dropna().sum()

print("\n")
print(f"Demande totale de chaleur : {total_heat_demand_MWh:.2f} MWh")
print(f"Consommation totale d'électricité : {total_electricity_consumption_MWh:.2f} MWh")

# Bilan : des emissions associées à la chaleur et l'électricité

total_emissions_co2_elec_t = (df["Electricity_Consumption[kW]"] * df["Taux de Co2"]).sum() / 1_000_000  # en tonne
total_emissions_co2_heat_t = (df["Heat_demand[kW]"] * df["cciag_co2"]).sum() / 1_000_000

print("\n")
print(f"Émissions de CO₂ dues à l'électricité : {total_emissions_co2_elec_t:.2f} tonnes")
print(f"Émissions de CO₂ dues à la chaleur (CCIAG) : {total_emissions_co2_heat_t:.2f} tonnes")

# Bilan : des coûts associés à la chaleur et l'électricité

total_cost_elec = (df["Electricity_Consumption[MW]"] * df["cost_MWh"]).dropna().sum()

total_cost_heat = (df["Heat_demand[MW]"] * df["cciag_cost"]).dropna().sum()

print("\n")
print(f"Coût total de l'électricité : {total_cost_elec:,.2f} €")
print(f"Coût total de la chaleur (CCIAG) : {total_cost_heat:,.2f} €")


--- BILAN DES CONSOMMATIONS, EMISSIONS ET COUTS du LNCMI actuel ---


Demande totale de chaleur : 2768.12 MWh
Consommation totale d'électricité : 13773.80 MWh


Émissions de CO₂ dues à l'électricité : 434.19 tonnes
Émissions de CO₂ dues à la chaleur (CCIAG) : 202.61 tonnes


Coût total de l'électricité : 893,860.77 €
Coût total de la chaleur (CCIAG) : 237,841.11 €


### Scenario 2 : REF HEBDO

First, definition of the rearrange() function, which allows the initial schedule to be reconstructed from the weekly blocks optimized by OMEGAlpes.

In [13]:
import traceback
def rearrange (df:pd.DataFrame , results:pd.DataFrame ,temp : pd.DataFrame  ):
    # df : original dataframe with high res data
    # results : dataframe results from omegalpes
    # temp: resampled dataset used as input for omagalpes
    
    t_step = temp.index[1] - temp.index[0]

    layout = {
    "font":{"size" : 21},
    
    "template": "plotly_white",
    "title" : "Optimal Planning for LNCMI (weekly) ",
    "width":1200,
    "height":800,

    'xaxis': {"showline": False,"linecolor": "black", "linewidth": 2,"tickangle" : -45,
            "title_text" : "Date","tickangle" : 45
             },

    "yaxis": {"showline": True,"linecolor": "black", "linewidth": 2,
              'zerolinecolor':'black',
               "side": 'left',
              "title_text" : "Power [MW] ",
    },
    "legend" : {"tracegroupgap":8 ,"font_size": 21, "orientation":"h", "yanchor":"bottom",
    "y":-0.35,"x":0.120, "title" : ""}
}

    df2 = pd.DataFrame({"Original Demand [MWh]" : df["Electricity_Consumption[MW]"],
                        "Rearranged Demand[MWh]" : [0]*len(df),
                        "co2": df["Taux de Co2"], "cost": df["cost_MWh"]},
                        index = df.index)

    df2 = df2.astype(float)

    for block in results.columns:

        try:
            nm = int (block.split("_")[2])
            s,e = temp.index[nm-1] , temp.index[nm]  - pd.Timedelta(seconds=60)
           
            if len(results[results[block] > 0.1].index)>0:
                ind = results[results[block] > 0.1].index[0]

                             
                start = temp.index[ind]
                if ind == results.index[-1]:
                    print (f"ind = {ind} , block : {block}")
                    end = temp.index[ind]  + t_step - pd.Timedelta(seconds=60)
                    
                else:
                    end =  temp.index[ind+1]  - pd.Timedelta(seconds=60)
                nm = int (block.split("_")[2])
                s = temp.index[nm] 

                if nm < len(temp) -1:
                    e = temp.index[nm+1]  - pd.Timedelta(seconds=60)
                else:
                    e = s  + t_step - pd.Timedelta(seconds=60)
         
                
                df2.loc[start:end , 'Rearranged Demand[MWh]'] = df2.loc[s:e , 'Original Demand [MWh]'].fillna(0).astype(float).tolist()

        except Exception as en:
            
            print (en)
            print(f"name : {block}")
            traceback.print_exc()
            print(s,e)
            print(start, end)
            print(len(df2.loc[start:end , 'Rearranged Demand[MWh]']) , len(df2.loc[s:e , 'Original Demand [MWh]'].tolist()))
            
    fig = px.line(df2)
    return df2


In [14]:
# --- 1. Initialisation des paramètres temporels ---

ts = "7d" # Période de regroupement (hebdomadaire)
start = str(df.resample(ts,label="left").mean().index[0])
end = "2024-09-11 00:00"
end = str(df.resample(ts,label = "left").mean().index[-1])
dt = 24 * 7  # 7 jours en heures
objectives =["co2"]
# weight = 10 

time = TimeUnit(start = start, end = end,dt = dt )

# --- 2. Préparation des données agrégées ---

temp = df[["Electricity_Consumption[MW]"]].copy().resample("1h",label = "left").mean().resample(ts,label = "left").sum()
temp["co2"] = df["Taux de Co2"].copy().resample(ts,label = "left").mean()
temp["cost_kwh"] = df["cost_kwh"].copy().resample(ts,label = "left").mean()
temp["Heat_demand[kW]"] = df["Heat_demand[kW]"].copy().resample(ts, label="left").mean()
temp["cciag_co2"] = df["cciag_co2"].copy().resample(ts, label="left").mean()
temp.fillna(0, inplace=True)

# --- 3. Création du modèle et du noeud énergetique ---

model = OptimisationModel(name='lncmi_model', time=time)
main_node = EnergyNode(time=time , name = "main_node",energy_type=elec)

# --- 4. Création dynamique des blocs de consommation ---

i = 0
dd = {}
var_ids = []

for period in temp.index:
    value = temp.loc[period,"Electricity_Consumption[MW]"] * 1000 # multiply by 1000 to change from MW to KW
    name = f"consumption_block_{i}"

    if "-08-" in str(period):
        print("hello")
        temp["new"] = [0] * len(temp)
        temp.loc[period,"new"] = value/time.DT
        value = temp["new"].tolist()

        dd[name] = value

        cmd = name + " = FixedConsumptionUnit(name=name , time=time,p = value, energy_type=elec )"

        exec(cmd)
 
    else:
        var_ids.append(i)
        cmd =  name + " = VariableConsumptionUnit(time = time, name = '" + name + \
                "', p_min= 0 , p_max=value/time.DT, " \
                "e_min=value, e_max=value, verbose=True, energy_type=elec)"
        exec(cmd)

        cmd = "cst = DefinitionConstraint(name ='SingleUse', exp='lpSum(" +  name +"_u[t] for t in time.I) <= 1 ', parent = " + name + ")"
        exec(cmd)

        cmd = "setattr(" + name + ", '" + name +"_SingleUse', cst)"
        exec(cmd)
        
    cmd = f"main_node.connect_units({name})"
    exec(cmd)

    i += 1 

# --- 5. Définition des puissances max et min ---

temp["p_max"] = [temp["Electricity_Consumption[MW]"].max()*1000] * len(temp)
temp["p_min"] = [0] * len(temp)

for ind in temp.index:
    if ind.month == 8:
        temp.loc[ind , "p_max"] = (temp.loc[ind , "Electricity_Consumption[MW]"]*1000)/time.DT
        temp.loc[ind , "p_min"] = (temp.loc[ind , "Electricity_Consumption[MW]"]*1000)/time.DT

# --- 6. Déclaration de l'unité de production ---

grid = VariableProductionUnit(time=time, p_max=temp.p_max.tolist() ,p_min = temp.p_min.tolist(), 
                            name = "grid_import" , energy_type=elec,co2_out=temp["co2"].tolist()[:],
                            operating_cost=temp["cost_kwh"].tolist()[:])    


# --- 7. Contrainte globale “SingleUnit”

exp = ""
for j in var_ids:
    exp += f"consumption_block_{j}_u[t]"
    if j < i-1:
        exp += " + "        
exp += " <= 1"

cst = DefinitionDynamicConstraint(name = "SingleUnit", t_range='for t in time.I',exp_t=exp, parent = None)
main_node.connect_units(grid)
setattr(main_node, 'SingleUnit', cst)

# --- 8. Définition de l'objectif ---

grid.minimize_co2_emissions()

# --- 9. Construction et résolution du modèle ---

model.add_nodes(main_node)
print ("READY TO SOLVE")
model.writeLP("test.lp")
model.solve_and_update(solver=GUROBI_CMD())
clear_output(wait=True)

# --- 10. Récupération et traitement des résultats ---

t = pd.DataFrame()
for j in range (i):
    cmd = f"t['consumption_block_{j}'] = consumption_block_{j}.p.get_value()"
    exec (cmd)

t = (t * time.DT)/1000  # kWh -> MWh
t["rearranged [MWh]"] = t.sum(axis = 1)
temp ["rearranged [MWh]"] = t["rearranged [MWh]"].tolist()

# --- 11. Visualisation des résultats hebdomadaires ---

import plotly.graph_objects as go

layout = {
    "font": {"size": 15},
    "template": "plotly_white",
    "title": "Weekly Rescheduling - [Objective: CO₂]",
    "width": 1200,
    "height": 500,
    "margin": {"b": 110},  # un peu plus bas pour éviter le chevauchement
    "xaxis": {
        "showline": False,  # Retire la ligne en dessous
        "zeroline": False,  # Supprime la ligne zéro éventuelle
        "tickangle": 45,
        "title_text": "Date",
        "type": "date",
        "title_font": {"color": "black", "size": 16}
    },
    "yaxis": {
        "showline": True,
        "linecolor": "black",
        "linewidth": 2,
        "zerolinecolor": 'black',
        "side": 'left',
        "title":{"text": "Energy [MWh]",
                "font": {"color": "black", "size": 16}},
        "tickfont": {"color": "black"}
    },
    "yaxis2": {
        "title": "Intensité Carbone [gCO₂/kWh]",
        "overlaying": "y",
        "side": "right",
        "showgrid": False,
        "linecolor": "black",
        "linewidth": 2,
        "tickfont": {"color": "black"},
       "title":{"text": "",
                "font": {"color": "black", "size": 16}},
    },
    "legend": {
        "tracegroupgap": 8,
        "font_size": 14,
        "orientation": "h",
        "yanchor": "top",
        "y": -0.35,  # Plus bas pour éviter chevauchement
        "x": 0.1,
        "title": ""
    },
    "barmode": "group"
}

fig1 = go.Figure()

# Demande originale
fig1.add_trace(go.Bar(
    x=temp.index,
    y=temp["Electricity_Consumption[MW]"],
    name="Original Demand",
    marker_color="#A9A9A9"
))

# Demande réarrangée
fig1.add_trace(go.Bar(
    x=temp.index,
    y=temp["rearranged [MWh]"],
    name="Replanned Demand",
    marker_color="#636EFA"
))

# Facteur d'émission CO₂ (électricité)
fig1.add_trace(go.Scatter(
    x=temp.index,
    y=temp["co2"],
    mode='lines+markers',
    name=" CO₂ Factor- Electric",
    line=dict(color='rgba(106, 13, 173, 0.6)', width=1.5, dash='dot'),
    marker=dict(size=5, color='#6a0dad'),
    yaxis='y2'
))

fig1.update_layout(layout)
fig1.show()




rearranged = rearrange(df=df, results=t[t.columns[:-1]], temp=temp)
df["Electricity_Consumption_rearranged[MW]"] = rearranged['Rearranged Demand[MWh]']

elec_initial_total = round(df["Electricity_Consumption[MW]"].dropna().sum(), 2)
elec_optimized_total = round(df["Electricity_Consumption_rearranged[MW]"].dropna().sum(), 2)
cciag_initial_total = round(df["Heat_demand[MW]"].dropna().sum(), 2)

elec_emissions_before = round((df["Electricity_Consumption[kW]"] * df["Taux de Co2"]).sum() / 1_000_000, 2)  # en tonnes
elec_emissions_after = round((df["Electricity_Consumption_rearranged[MW]"] * df["Taux de Co2"]).sum() / 1_000, 2)
reduction_emissions_elec = round(100 * (1 - elec_emissions_after / elec_emissions_before), 2)

heat_emissions = round((df["Heat_demand[kW]"] * df["cciag_co2"]).sum() / 1_000_000, 2)

total_emissions_before = round(elec_emissions_before + heat_emissions, 2)
total_emissions_after  = round(elec_emissions_after + heat_emissions, 2)
total_reduction = round(100 * (1 - total_emissions_after / total_emissions_before), 2)


elec_cost_before = round((df["Electricity_Consumption[kW]"] * df["cost_MWh"]/ 1_000).dropna().sum())
elec_cost_after = round((df["Electricity_Consumption_rearranged[MW]"] * df["cost_MWh"]).dropna().sum())
heat_cost = round((df["Heat_demand[MW]"] * df["cciag_cost"]).dropna().sum())

# --- DISPLAY ---
print("\n--- CONSUMPTION ---")
print(f"ELECTRICITY consumption BEFORE optimization : {elec_initial_total} MWh")
print(f"ELECTRICITY consumption AFTER optimization  : {elec_optimized_total} MWh")
print(f"CCIAG HEAT consumption BEFORE optimization  : {cciag_initial_total} MWh")

print("\n--- CO2 EMISSIONS (in tonnes) ---")
print(f"ELECTRICITY emissions BEFORE optimization : {elec_emissions_before} tCO₂")
print(f"ELECTRICITY emissions AFTER optimization  : {elec_emissions_after} tCO₂")
print(f"→ Reduction in ELECTRICITY-related emissions : {reduction_emissions_elec} %")

print(f"CCIAG HEAT emissions : {heat_emissions} tCO₂")

print("\n--- TOTAL EMISSIONS ---")
print(f"TOTAL emissions BEFORE optimization : {total_emissions_before} tCO₂")
print(f"TOTAL emissions AFTER optimization  : {total_emissions_after} tCO₂")
print(f"→ TOTAL CO2 emissions reduction : {total_reduction} %")

print("\n--- ECONOMIC BALANCE ---")
print(f"Electricity cost BEFORE optimization : {elec_cost_before:,.2f} €")
print(f"Electricity cost AFTER optimization  : {elec_cost_after:,.2f} €")
print(f"Heat cost AFTER optimization         : {heat_cost:,.2f} €")
print(f"TOTAL cost BEFORE optimization       : {elec_cost_before + heat_cost:,.2f} €")
print(f"TOTAL cost AFTER optimization        : {elec_cost_after + heat_cost:,.2f} €")


ind = 51 , block : consumption_block_46

--- CONSUMPTION ---
ELECTRICITY consumption BEFORE optimization : 13773.8 MWh
ELECTRICITY consumption AFTER optimization  : 13773.8 MWh
CCIAG HEAT consumption BEFORE optimization  : 2768.12 MWh

--- CO2 EMISSIONS (in tonnes) ---
ELECTRICITY emissions BEFORE optimization : 434.19 tCO₂
ELECTRICITY emissions AFTER optimization  : 349.53 tCO₂
→ Reduction in ELECTRICITY-related emissions : 19.5 %
CCIAG HEAT emissions : 202.61 tCO₂

--- TOTAL EMISSIONS ---
TOTAL emissions BEFORE optimization : 636.8 tCO₂
TOTAL emissions AFTER optimization  : 552.14 tCO₂
→ TOTAL CO2 emissions reduction : 13.29 %

--- ECONOMIC BALANCE ---
Electricity cost BEFORE optimization : 893,861.00 €
Electricity cost AFTER optimization  : 876,826.00 €
Heat cost AFTER optimization         : 237,841.00 €
TOTAL cost BEFORE optimization       : 1,131,702.00 €
TOTAL cost AFTER optimization        : 1,114,667.00 €


### Scenario 3 : VALO HEBDO

First step in modeling: Rescheduling the weeks of experimentation.

Before creating the model, we modify the ElectricalToThermalConversionUnit class so that it can be used as a shiftable unit.

In [15]:
class ElectricalToThermalConversionUnit(ConversionUnit):
    """
    **Description**
        DEPRECATED: please now use SingleConversionUnit with relevant energy
        types

        Electrical to thermal Conversion unit with an electricity consumption
        and a thermal production

    **Attributes**

     * thermal_production_unit : thermal production unit (thermal output)
     * elec_consumption_unit : electricity consumption unit (electrical
       input)
     * conversion : Definition Dynamic Constraint linking the electrical
     input to
       the thermal output through the electrical to thermal ratio

    """

    def __init__(self, time, name, pmin_in_elec=0, pmax_in_elec=1e+5,
                 p_in_elec=None, pmin_out_therm=0, pmax_out_therm=1e+5, shiftable = False,
                 p_out_therm=None, elec_to_therm_ratio=1,e_in_max = None, 
                 e_in_min= None, verbose=True):

        """
        :param time: TimeUnit describing the studied time period
        :param name: name of the electrical to thermal conversion unit
        :param pmin_in_elec: minimal incoming electrical power
        :param pmax_in_elec: maximal incoming electrical power
        :param p_in_elec: power input for the electrical consumption unit
        :param pmin_out_therm: minimal power output (thermal)
        :param pmax_out_therm: maximal power output (thermal)
        :param p_out_therm: power output (thermal)
        :param elec_to_therm_ratio: electricity to thermal ratio <=1
        """

        if p_out_therm is None:
            self.thermal_production_unit = VariableProductionUnit(
                time, name + '_therm_prod', energy_type='Thermal',
                p_min=pmin_out_therm, p_max=pmax_out_therm,
                verbose=verbose)
        else:
            self.thermal_production_unit = FixedProductionUnit(
                time, name + '_therm_prod', energy_type='Thermal',
                p=p_out_therm, verbose=verbose)

        if p_in_elec is None and shiftable == False :
            self.elec_consumption_unit = VariableConsumptionUnit(
                time, name + '_elec_cons', p_min=pmin_in_elec,
                p_max=pmax_in_elec, energy_type='Electrical', e_max=e_in_max,
                e_min = e_in_min , verbose=verbose)
            
        elif p_in_elec is not None and shiftable :
            self.elec_consumption_unit = ShiftableConsumptionUnit(
                time, name + '_elec_cons', power_values=p_in_elec,
                energy_type='Electrical', mandatory=True , verbose=verbose)
        else:
            self.elec_consumption_unit = FixedConsumptionUnit(
                time, name + '_elec_cons', p=p_in_elec,
                energy_type='Electrical', verbose=verbose)

        ConversionUnit.__init__(self, time, name,
                                prod_units=[self.thermal_production_unit],
                                cons_units=[self.elec_consumption_unit])

        if isinstance(elec_to_therm_ratio, (int, float)):  # e2h_ratio is a
            # mean value
            if elec_to_therm_ratio <= 1:
                self.conversion = DefinitionDynamicConstraint(
                    exp_t='{0}_p[t] == {1} * {2}_p[t]'.format(
                        self.thermal_production_unit.name,
                        elec_to_therm_ratio,
                        self.elec_consumption_unit.name),
                    t_range='for t in time.I', name='conversion', parent=self)
            else:
                raise ValueError('The elec_to_therm_ratio should be lower '
                                 'than 1 (therm_production<elec_consumption)')

        elif isinstance(elec_to_therm_ratio, list):  # e2h_ratio is a list of
            # values
            if len(elec_to_therm_ratio) == self.time.LEN:  # it must have the
                #  right size, i.e. the TimeUnit length.
                if all(e <= 1 for e in elec_to_therm_ratio):
                    self.conversion = DefinitionDynamicConstraint(
                        exp_t='{0}_p[t] == {1}[t] * {2}_p[t]'.format(
                            self.thermal_production_unit.name,
                            elec_to_therm_ratio,
                            self.elec_consumption_unit.name),
                        t_range='for t in time.I', name='conversion',
                        parent=self)
                else:
                    raise ValueError(
                        'The elec_to_therm_ratio values should be '
                        'lower than 1 (therm_production<elec_'
                        'consumption)')
            else:
                raise IndexError('The length of the elec_to_therm_ratio '
                                 'vector should be of the same length as the '
                                 'TimeUnit of the studied period')

        elif isinstance(elec_to_therm_ratio, dict):  # e2h_ratio is a dict of
            # values
            if len(elec_to_therm_ratio) == self.time.LEN:
                if all(e <= 1 for e in elec_to_therm_ratio.values()):
                    self.conversion = DefinitionDynamicConstraint(
                        exp_t='{0}_p[t] == {1}[t] * {2}_p[t]'.format(
                            self.thermal_production_unit.name,
                            elec_to_therm_ratio,
                            self.elec_consumption_unit.name),
                        t_range='for t in time.I', name='conversion',
                        parent=self)
                else:
                    raise ValueError(
                        'The elec_to_therm_ratio values should be '
                        'lower than 1 (therm_production<elec_'
                        'consumption)')
            else:
                raise IndexError('The length of the elec_to_therm_ratio '
                                 'dictionary should be of the same length as '
                                 'the TimeUnit of the studied period')
        else:
            raise TypeError(
                "Electricity to thermal ratio should be a mean value or a "
                "vector (list or dict) on the whole time period !")


In [27]:
# --- 1. Initialisation des paramètres temporels ---

ts = "7d"  # Période de regroupement (hebdomadaire)
start = str(df.resample(ts, label="left").mean().index[0])
end = str(df.resample(ts, label="left").mean().index[-1])
dt = 24 * 7  # 7 jours en heures
objectives = ["co2"]
# weight = 10

time = TimeUnit(start=start, end=end, dt=dt)

# --- 2. Préparation des données agrégées ---

temp = df[["Electricity_Consumption[MW]"]].copy().resample("1h", label="left").mean().resample(ts, label="left").sum()
temp["co2"] = df["Taux de Co2"].copy().resample(ts, label="left").mean()
temp["cciag_co2"] = df["cciag_co2"].copy().resample(ts, label="left").mean()
temp["cost_MWh"] = df["cost_MWh"].copy().resample(ts, label="left").mean()
temp["Heat_demand[kW]"] = df["Heat_demand[kW]"].copy().resample(ts, label="left").mean()
temp.fillna(0, inplace=True)

# --- 3. Création du modèle et des nœuds d'énergie ---

model = OptimisationModel(name='lncmi_model', time=time)
elec_node = EnergyNode(time=time, name="elec_node", energy_type=elec)
heat_node = EnergyNode(time=time, name="heat_node", energy_type=thermal)
heat_node_2 = EnergyNode(time=time, name="heat_node_2", energy_type=thermal)

# --- 4. Création dynamique des blocs de consommation ---

i = 0
dd = {}
var_ids = []

for period in temp.index:
    value = temp.loc[period, "Electricity_Consumption[MW]"] * 1000  # MW -> kW
    name = f"consumption_block_{i}"
    var_ids.append(i)

    cmd = name + " = ElectricalToThermalConversionUnit(time = time, name = '" + name + \
          "',elec_to_therm_ratio = 0.85,  pmin_in_elec= 0 , pmax_in_elec = value/time.DT, " \
          " p_in_elec = None, pmin_out_therm = 0, pmax_out_therm = value/time.DT, "\
          "e_in_min=value, e_in_max=value, verbose=True)"
    exec(cmd)

    cmd = "cst = DefinitionConstraint(name ='SingleUse', exp='lpSum(" + name + "_elec_cons_u[t] for t in time.I) <= 1 ', parent = " + name + ")"
    exec(cmd)

    cmd = "setattr(" + name + ", '" + name + "_SingleUse', cst)"
    exec(cmd)

    exec(f"elec_node.connect_units({name}.elec_consumption_unit  )")
    exec(f"heat_node.connect_units({name}.thermal_production_unit )")

    i += 1

# --- 5. Définition des puissances max et min ---

temp["p_max"] = [temp["Electricity_Consumption[MW]"].max() * 1000] * len(temp)
temp["p_min"] = [0] * len(temp)

for ind in temp.index:
    if ind.month == 8:
        temp.loc[ind, "p_max"] = (temp.loc[ind, "Electricity_Consumption[MW]"] * 1000) / time.DT
        temp.loc[ind, "p_min"] = (temp.loc[ind, "Electricity_Consumption[MW]"] * 1000) / time.DT

# --- 6. Déclaration des unités de production/consommation ---

cciag = VariableProductionUnit(time=time, name="cciag", p_min=0, p_max=10000, co2_out=temp["cciag_co2"].tolist(), operating_cost=100, energy_type=thermal)
reject = VariableConsumptionUnit(time=time, name="rejected_heat", p_max=temp.p_max.max(), energy_type=thermal)
cnrs_heat = FixedConsumptionUnit(time=time, name="CNRS_heat", p=temp["Heat_demand[kW]"].tolist(), energy_type=thermal)
grid = VariableProductionUnit(time=time, p_max=temp.p_max.tolist(), p_min=temp.p_min.tolist(), 
                               name="grid_import", energy_type=elec, co2_out=temp["co2"].tolist(), 
                               operating_cost=temp["cost_MWh"].tolist())

# constraint to define the overconsumption of the magnets when heatrecovery is activated 
# The overconsumption unit is an extra consumption unit that simulates the over consumption and should be added to the final consumption of the various consumption profile

over_consumption = VariableConsumptionUnit(time=time, name="over_consumption", p_max=temp.p_max.max(), energy_type=elec)
exp = f" + 0.045 * (CNRS_heat_p[t] - cciag_p[t]) == over_consumption_p[t]"                                                     # attention car le 0.045 doit etre multiplié à la puissance de l'aimant à ce moment la et non à l'energie recupérée
cst = DefinitionDynamicConstraint(name="over_consumption_cst", t_range='for t in time.I', exp_t=exp, parent=over_consumption)
setattr(over_consumption, 'over_consumption_cst', cst)

# --- 7. Connexions entre les nœuds ---

elec_node.connect_units(over_consumption)
elec_node.connect_units(grid)
heat_node_2.connect_units(cnrs_heat, cciag)
heat_node.connect_units(reject)
heat_node.export_to_node(heat_node_2)

# --- 8. Ajout d'une contrainte dynamique unique ---

exp = " + ".join([f"consumption_block_{j}_elec_cons_u[t]" for j in var_ids]) + " <= 1"
cst = DefinitionDynamicConstraint(name="SingleUnit", t_range='for t in time.I', exp_t=exp, parent=None)
setattr(elec_node, 'SingleUnit', cst)

# --- 9. Définition de l'objectif ---

if "co2" in objectives:
    grid.minimize_co2_emissions()
    cciag.minimize_co2_emissions()

# --- 10. Construction et résolution du modèle ---

model.add_nodes(elec_node, heat_node, heat_node_2)
print("READY TO SOLVE")
model.writeLP("test.lp")
model.solve_and_update(solver=GUROBI_CMD())
clear_output(wait=True)

# --- 11. Récupération et traitement des résultats ---

t = pd.DataFrame()
for j in range(i):
    cmd = f"t['consumption_block_{j}'] = consumption_block_{j}.elec_consumption_unit.p.get_value()"
    exec(cmd)

t = (t * time.DT)/1000  # kWh -> MWh
t["rearranged [MWh]"] = t.sum(axis=1)
temp["rearranged [MWh]"] = t["rearranged [MWh]"].tolist()

# --- 11b. Récupération de la production de chaleur de la CCIAG avant et après replanification ---

cciag_heat = pd.Series(cciag.p.get_value(), index=temp.index)  # en kW   # Récupération de la puissance de chaleur produite par la CCIAG à chaque pas de temps
cciag_heat_weekly = (cciag_heat * time.DT) / 1000  # kWh → MWh (DT = 168h pour 1 semaine)  # Conversion en MWh par semaine
temp["CCIAG_heat_output[MWh]"] = cciag_heat_weekly.tolist()  # Ajout dans le DataFrame temp

temp["CCIAG_heat_output_initial[MWh]"] = temp["Heat_demand[kW]"] * time.DT / 1000  # kWh → MWh  # Demande initiale : toute la demande CNRS était fournie par la CCIAG

# Affichage des résultats

#print(temp.columns)

# --- CONSOMMATIONS ÉLECTRIQUES
elec_initial_total = round(temp["Electricity_Consumption[MW]"].sum(), 2)
elec_optimized_total = round(temp["rearranged [MWh]"].sum(), 2)

# --- ÉMISSIONS ÉLECTRIQUES (en tCO₂)
elec_emissions_before = round((temp["Electricity_Consumption[MW]"] * temp["co2"] / 1000).sum(), 2)
elec_emissions_after  = round((temp["rearranged [MWh]"] * temp["co2"] / 1000).sum(), 2)
reduction_emissions_elec = round(100 * (1 - elec_emissions_after / elec_emissions_before), 2)

# --- CONSOMMATIONS THERMIQUES
cciag_initial_total = round(temp["CCIAG_heat_output_initial[MWh]"].sum(), 2)
cciag_optimized_total = round(temp["CCIAG_heat_output[MWh]"].sum(), 2)

# --- ÉMISSIONS THERMIQUES (en tCO₂)
heat_emissions_before = round((temp["CCIAG_heat_output_initial[MWh]"] * temp["cciag_co2"] / 1000).sum(), 2)
heat_emissions_after  = round((temp["CCIAG_heat_output[MWh]"] * temp["cciag_co2"] / 1000).sum(), 2)
reduction_emissions_heat = round(100 * (1 - heat_emissions_after / heat_emissions_before), 2)

# --- ÉMISSIONS TOTALES
total_emissions_before = round(elec_emissions_before + heat_emissions_before, 2)
total_emissions_after  = round(elec_emissions_after + heat_emissions_after, 2)
total_reduction = round(100 * (1 - total_emissions_after / total_emissions_before), 2)

# --- AFFICHAGE ---
print("\n--- CONSUMPTION ---")
print("\n--- CONSOMMATIONS ---")
print(f"ELECTRICITY consumption BEFORE optimization: {elec_initial_total} MWh")
print(f"ELECTRICITY consumption AFTER optimization: {elec_optimized_total} MWh")
print(f"CCIAG HEAT consumption BEFORE optimization: {cciag_initial_total} MWh")
print(f"CCIAG HEAT consumption AFTER optimization: {cciag_optimized_total} MWh")

print("\n--- CO2 EMISSIONS (in tons) ---")
print(f"ELECTRICITY emissions BEFORE optimization: {elec_emissions_before} tCO₂")
print(f"ELECTRICITY emissions AFTER optimization: {elec_emissions_after} tCO₂")
print(f"Reduction in ELECTRICITY-related emissions: {reduction_emissions_elec} %")

print(f"CCIAG HEAT emissions BEFORE optimization: {heat_emissions_before} tCO₂")
print(f"CCIAG HEAT emissions AFTER optimization: {heat_emissions_after} tCO₂")
print(f"Reduction in HEAT emissions: {reduction_emissions_heat} %")

print("\n--- TOTAL EMISSIONS ---")
print(f"TOTAL EMISSIONS BEFORE optimization: {total_emissions_before} tCO₂")
print(f"TOTAL EMISSIONS AFTER optimization: {total_emissions_after} tCO₂")
print(f"TOTAL reduction in CO2 emissions: {total_reduction} %")


# --- 12. Visualisation des résultats hebdomadaires ---

import plotly.graph_objects as go

layout = {
    "font": {"size": 18},
    "template": "plotly_white",
    "title": "",
    "width": 1300,
    "height": 700,
    "margin": {"b": 110},  # un peu plus bas pour éviter le chevauchement
    "xaxis": {
        "showline": False,  # Retire la ligne en dessous
        "zeroline": False,  # Supprime la ligne zéro éventuelle
        "tickangle": 45,
        "title_text": "Date",
        "type": "date",
        "title_font": {"color": "black", "size": 20}
    },
    "yaxis": {
        "showline": True,
        "linecolor": "black",
        "linewidth": 2,
        "zerolinecolor": 'black',
        "side": 'left',
        "title":{"text": "Energy [MWh]",
                "font": {"color": "black", "size": 20}},
        "tickfont": {"color": "black"}
    },
    "yaxis2": {
        "title": "Carbon Intensity [gCO₂/kWh]",
        "overlaying": "y",
        "side": "right",
        "showgrid": False,
        "linecolor": "black",
        "linewidth": 2,
        "tickfont": {"color": "black"},
        "title":{"text": "Carbon Intensity [gCO₂/kWh]",
                "font": {"color": "black", "size": 20}},
    },
    "legend": {
        "tracegroupgap": 8,
        "font_size": 18,
        "orientation": "h",
        "yanchor": "top",
        "y": -0.35,  # Plus bas pour éviter chevauchement
        "x": 0.1,
        "title": ""
    },
    "barmode": "group"
}

fig1 = go.Figure()

# Demande originale
fig1.add_trace(go.Bar(
    x=temp.index,
    y=temp["Electricity_Consumption[MW]"],
    name="Originale Demand",
    marker_color="#A9A9A9"
))

# Demande réarrangée
fig1.add_trace(go.Bar(
    x=temp.index,
    y=temp["rearranged [MWh]"],
    name="Rearranged Demand",
    marker_color="#636EFA"
))

# Facteur d'émission CO₂ (électricité)
fig1.add_trace(go.Scatter(
    x=temp.index,
    y=temp["co2"],
    mode='lines+markers',
    name="CO₂ Factor - Elec",
    line=dict(color='rgba(106, 13, 173, 0.6)', width=1.5, dash='dot'),
    marker=dict(size=5, color='#6a0dad'),
    yaxis='y2'
))

# Facteur d'émission CO₂ - CCIAG
fig1.add_trace(go.Scatter(
    x=temp.index,
    y=temp["cciag_co2"],
    mode='lines+markers',
    name="CO₂ Factor - CCIAG",
    line=dict(color='rgba(0,100,0,0.6)', width=1.5, dash='dot'),
    marker=dict(size=5, color='darkgreen'),
    yaxis='y2'
))

# Demande de chaleur CNRS
fig1.add_trace(go.Scatter(
    x=temp.index,
    y=temp["CCIAG_heat_output_initial[MWh]"],
    mode='lines',
    name="CNRS Heat Demand",
    line=dict(color='red', width=1.5)
))

fig1.update_layout(layout)
fig1.show()

# --- 13 Collecte du profile réarangé au pas de temps fin  ---

rearranged = rearrange(df=df, results=t[t.columns[:-1]], temp=temp)
df["Electricity_Consumption_rearranged[MW]"] = rearranged['Rearranged Demand[MWh]']


# check si rearrange a bien fonctionné : 
total_electricity_Consumption_rearranged_MW = df["Electricity_Consumption_rearranged[MW]"].dropna().sum()
print("\n")
print(f"Total electricity consumption optimal planning : {total_electricity_Consumption_rearranged_MW:.2f} MWh")


--- CONSUMPTION ---

--- CONSOMMATIONS ---
ELECTRICITY consumption BEFORE optimization: 13773.8 MWh
ELECTRICITY consumption AFTER optimization: 13773.8 MWh
CCIAG HEAT consumption BEFORE optimization: 2768.12 MWh
CCIAG HEAT consumption AFTER optimization: 128.04 MWh

--- CO2 EMISSIONS (in tons) ---
ELECTRICITY emissions BEFORE optimization: 437.31 tCO₂
ELECTRICITY emissions AFTER optimization: 379.35 tCO₂
Reduction in ELECTRICITY-related emissions: 13.25 %
CCIAG HEAT emissions BEFORE optimization: 203.67 tCO₂
CCIAG HEAT emissions AFTER optimization: 7.63 tCO₂
Reduction in HEAT emissions: 96.25 %

--- TOTAL EMISSIONS ---
TOTAL EMISSIONS BEFORE optimization: 640.98 tCO₂
TOTAL EMISSIONS AFTER optimization: 386.98 tCO₂
TOTAL reduction in CO2 emissions: 39.63 %


ind = 51 , block : consumption_block_46


Total electricity consumption optimal planning : 13773.80 MWh


2eme modèle : Optimisation de la gestion du stockage thermique.

In [17]:
def waste_heat_rec(dt=1, start_date='2021-01-18 00:00:00', end_date="2021-01-28 23:59:00", indus_cons=[], heat_load=[],  elec2therm_ratio=0.85, pc_max=5000, pd_max=5000,
                   pc_min=1000, pd_min=1000, capa=20000, cop_hp=3,
                   pmax_elec_hp=1000, storage_soc_0=0.2, p_thresh_reco= 0, write_lp=True):
    
    # OPTIMIZATION MODEL
    # Creating the unit dedicated to time management
    time = TimeUnit(dt= dt, start=start_date, end=end_date)

    # Creating an empty model
    model = OptimisationModel(time=time, name='waste_heat_recovery_model')

    # Creating the electro-intensive industry unit
    indus_cons = [0 if val < p_thresh_reco else val for val in indus_cons]      # Remplace les valeurs de indus_cons inferieur à p_thresh_reco par zéro
    indus = ElectricalToThermalConversionUnit(time, 'indus',
                                              elec_to_therm_ratio=
                                              elec2therm_ratio,
                                              p_in_elec=indus_cons)

    # Creating unit for heat dissipation from the industrial process
    dissipation = VariableConsumptionUnit(time, 'dissipation',
                                          energy_type='Thermal')

    # Creating the thermal storage
    thermal_storage = StorageUnit(time, 'thermal_storage', pc_max=pc_max,
                                  pd_max=pd_max, pc_min=pc_min, pd_min=pd_min,
                                  capacity=capa, e_0=storage_soc_0 * capa)

    # Creating the heat pump
    heat_pump = HeatPump(time, 'heat_pump', cop=cop_hp,
                         pmax_in_elec=pmax_elec_hp)

    # Creating the district heat load
    district_heat_load = FixedConsumptionUnit(time, 'district_heat_load',
                                              p=heat_load,
                                              energy_type='Thermal')

    # Creating the heat production plants
    heat_production = ProductionUnit(time, name='heat_production',
                                     energy_type='Thermal')

    # Creating the heat node for the energy flows
    heat_node_bef_valve = EnergyNode(time, 'heat_node_bef_valve',
                                     energy_type='Thermal')
    heat_node_aft_valve = EnergyNode(time, 'heat_node_aft_valve',
                                     energy_type='Thermal')
    heat_node_aft_hp = EnergyNode(time, 'heat_node_aft_hp',
                                  energy_type='Thermal')

    # Connecting units to the nodes
    heat_node_bef_valve.connect_units(indus.thermal_production_unit,
                                      dissipation)
    heat_node_bef_valve.export_to_node(
        heat_node_aft_valve)  # Export after the valve
    heat_node_aft_valve.connect_units(thermal_storage,
                                      heat_pump.thermal_consumption_unit)

    heat_node_aft_hp.connect_units(heat_pump.thermal_production_unit,
                                   heat_production, district_heat_load)

    # OBJECTIVE CREATION
    # Minimizing the part of the heat load covered by the heat production plant
    heat_production.minimize_production()

    # Adding all nodes (and connected units) to the optimization model
    model.add_nodes(heat_node_bef_valve, heat_node_aft_valve, heat_node_aft_hp)

    if write_lp:
        model.writeLP('waste_e_recovery.lp')
    # Writing into lp file
    model.solve_and_update(GUROBI_CMD())  # Running optimization and update values

    return model, time, indus, district_heat_load, heat_pump, \
           heat_production, dissipation, thermal_storage, \
           heat_node_bef_valve, heat_node_aft_valve, \
           heat_node_aft_hp


def print_results_waste_heat(model, time, indus, district_heat_load,
                             heat_pump, heat_production,dissipation, thermal_storage,
                             heat_node_bef_valve, heat_node_aft_valve,
                             heat_node_aft_hp):
    """
        *** This function prints the optimisation result:
                - The district consumption during the year
                - The industry consumption during the year
                - The district heat network production during the year
                - The heat exported from the industry
                - The rate of the load covered by the industry

            And plots the power curves :
            On the first figure : the energy out of the industry with the
            recovered and the dissipated parts
            On the second figure: the energy on the district heating network
            with the part produced by the heat pump and
            the part produced by the district heating production unit.

    """

    # Print results
    if LpStatus[model.status] == 'Optimal':
        print("\n - - - - - OPTIMIZATION RESULTS - - - - - ")
        print('Magnets electrical consumption = {0} GWh.'.format(
              round(indus.elec_consumption_unit.e_tot.value / 1e6, 2)))
        print('Dissipation consumption = {0} GWh.'.format(
              round(dissipation.e_tot.value / 1e6, 2)))
        print('Heat_pump heat consumption = {0} GWh.'.format(
              round(heat_pump.thermal_consumption_unit.e_tot.value / 1e6, 2)))
        print('Heat pump electricity consumption = {0} GWh.'.format(
              round(heat_pump.elec_consumption_unit.e_tot.value / 1e6, 2)))
        print('Heat_pump heat production = {0} GWh.'.format( 
              round(heat_pump.thermal_production_unit.e_tot.value / 1e6, 2)))
        print('CCIAG production = {0} GWh.'.format( 
              round(heat_production.e_tot.value/ 1e6, 2)))
        print('District consumption = {0} GWh.'.format(
            round(district_heat_load.e_tot.value / 1e6, 2)))
        print("% of the load coming from the heat pump = {0} %.".format(
            round(heat_pump.thermal_production_unit.e_tot.value /
                  district_heat_load.e_tot.value * 100)))
        print("% of the load coming from LNCMI heat = {0} %.".format(
            round(heat_pump.thermal_consumption_unit.e_tot.value /
                  district_heat_load.e_tot.value * 100)))
        # value is a dict, with time as a key, and power levels as values.

    elif LpStatus[model.status] == 'Infeasible':
        print("Sorry, the optimisation problem has no feasible solution !")

    elif LpStatus[model.status] == 'Unbounded':
        print("The cost function of the optimisation problem is unbounded !")

    elif LpStatus[model.status] == 'Undefined':
        print("Sorry, a feasible solution has not been found (but may exist). "
              "PuLP does not manage to interpret the solver's output, "
              "the infeasibility of the MILP problem may have been "
              "detected during presolve")

    else:
        print("Sorry, the optimisation problem has not been solved.")


In [18]:
# *** OPTIMIZATION PARAMETERS *** #

# --- Electricity-to-Thermal conversion ---
ELEC_TO_HEAT_RATIO = 0.85                                   # 85% of the electrical consumption is converted into heat (we are taking PMAGNET)

# --- Thermal storage parameters ---
CAPA_STORAGE = 7500                                        # Storage capacity of 10MWh
PC_MAX_STORAGE = PD_MAX_STORAGE = CAPA_STORAGE / 3          # The maximal charging and discharging powers both equal CAPA_STORAGE / 3
PC_MIN_STORAGE = PD_MIN_STORAGE = 0 * PC_MAX_STORAGE        # When charging/discharging, the power should at least be 0% of the maximal charging/discharging powers
SOC_0_STORAGE = 0.2                                         # Initial state of charge of 25%

# --- Heat pump parameters ---
COP = 4.5                                                   # The coefficient of performance equals 3
P_MAX_HP = 450                                             # The heat pump has a electrical power limit of 4.5 MW

# --- p_thresh_reco parameters ---
P_THRESH_RECO = 0                                           # P_THRESH_RECO = 0 MW

In [19]:
# --- Time parameters ---
DT=1
START_DATE="01/01/2018 00:00" 
END_DATE="30/12/2018 23:00"

# *** Importing time-dependent data from files *** #
INDUS_CONS = (df["Electricity_Consumption_rearranged[MW]"] * 1000).tolist()
HEAT_LOAD = df["Heat_demand[kW]"].tolist()

expected_energy_kWh = sum(INDUS_CONS)
expected_energy_GWh = expected_energy_kWh / 1e6
print(f"Expected magnet energy = {expected_energy_GWh:.2f} GWh")

# *** RUN MAIN ***
MODEL, TIME, INDUS, DISTRICT_HEAT_LOAD, HEAT_PUMP, HEAT_PRODUCTION, \
DISSIPATION, THERMAL_STORAGE, HEAT_NODE_BEF_VALVE, HEAT_NODE_AFT_VALVE, \
HEAT_NODE_AFT_HP = waste_heat_rec(dt=DT, start_date=START_DATE, end_date=END_DATE, indus_cons=INDUS_CONS, heat_load=HEAT_LOAD,
                                    elec2therm_ratio=ELEC_TO_HEAT_RATIO,
    pc_max=PC_MAX_STORAGE,
    pd_max=PD_MAX_STORAGE, pc_min=PC_MIN_STORAGE,
    pd_min=PD_MIN_STORAGE, capa=CAPA_STORAGE, cop_hp=COP,
    pmax_elec_hp=P_MAX_HP, storage_soc_0=SOC_0_STORAGE, p_thresh_reco=P_THRESH_RECO, write_lp=True)

Expected magnet energy = 13.77 GWh
You are studying the period from 2018-01-01 00:00:00 to 2018-12-30 23:00:00
Creating the indus_therm_prod.


Creating the indus_elec_cons.
Creating the indus.
Creating the dissipation.
Creating the thermal_storage_discharge.
Creating the thermal_storage_charge.
Creating the thermal_storage.
Creating the heat_pump_therm_prod.
Creating the heat_pump_therm_cons.
Creating the heat_pump_elec_cons.
Creating the heat_pump.
Creating the district_heat_load.
Creating the heat_production.
Creating the heat_node_bef_valve.
Creating the heat_node_aft_valve.
Creating the heat_node_aft_hp.

--- Adding all variables to the model ---
Adding variable : heat_node_bef_valve_energy_export_to_heat_node_aft_valve
Adding variable : heat_node_bef_valve_is_exporting_to_heat_node_aft_valve
Adding variable : indus_therm_prod_p
Adding variable : indus_therm_prod_e_tot
Adding variable : indus_therm_prod_u
Adding variable : dissipation_p
Adding variable : dissipation_e_tot
Adding variable : dissipation_u
Adding variable : indus_elec_cons_p
Adding variable : indus_elec_cons_e_tot
Adding variable : thermal_storage_charge_p
A

In [20]:
# *** SHOW THE RESULTS ***
print_results_waste_heat(MODEL, TIME, INDUS, DISTRICT_HEAT_LOAD, HEAT_PUMP,
                            HEAT_PRODUCTION, DISSIPATION, THERMAL_STORAGE,
                            HEAT_NODE_BEF_VALVE, HEAT_NODE_AFT_VALVE,
                            HEAT_NODE_AFT_HP)




 - - - - - OPTIMIZATION RESULTS - - - - - 
Magnets electrical consumption = 13.77 GWh.
Dissipation consumption = 10.39 GWh.
Heat_pump heat consumption = 1.32 GWh.
Heat pump electricity consumption = 0.38 GWh.
Heat_pump heat production = 1.7 GWh.
CCIAG production = 1.07 GWh.
District consumption = 2.77 GWh.
% of the load coming from the heat pump = 61 %.
% of the load coming from LNCMI heat = 48 %.


## Results

In [21]:
# Récupérer les puissances horaires de la production de chaleur de la CCIAG

cciag_prod_values = HEAT_PRODUCTION.p.value                                                # Extraction des puissances (en kW)
cciag_prod_series = pd.Series(                                                             # Créer une série temporelle avec les données récupérées (index chronologique)
    data=cciag_prod_values.values(),                                                       # On utilise .values() pour extraire la liste des valeurs
    index=pd.date_range(start="2018-01-01", periods=len(cciag_prod_values), freq="H")      # Index temporel (horaire)
)
df["CCIAG Production (kW)"] = cciag_prod_series.values                                     # Ajouter la série au DataFrame en utilisant les valeurs de la série


# Récupérer les puissances horaires de la consommation électrique de la pompe à chaleur

hp_elec_values = HEAT_PUMP.elec_consumption_unit.p.value                                   # Extraction des puissances (en kW)
hp_elec_series = pd.Series(                                                                # Créer une série temporelle avec les données récupérées (index chronologique)
    data=hp_elec_values.values(),                                                          # On utilise .values() pour extraire la liste des valeurs
    index=pd.date_range(start="2018-01-01", periods=len(hp_elec_values), freq="H")         # Index temporel (horaire)
)
df["Heat Pump Electricity Consumption (kW)"] = hp_elec_series.values                       # Ajouter la série au DataFrame en utilisant les valeurs de la série

# ---Bilan LNCMI actuel  ---

print("\n--- SUMMARY OF CONSUMPTION, EMISSIONS, AND COSTS of the current LNCMI ---")

# Bilan : de la consommation totale de chaleur et d'électricité en MWh 

total_heat_demand_MWh = df["Heat_demand[MW]"].dropna().sum()
total_electricity_consumption_MWh = df["Electricity_Consumption[MW]"].dropna().sum()

print(f"Total Heat Demand : {total_heat_demand_MWh:.2f} MWh")
print(f"Total Electricity Demand : {total_electricity_consumption_MWh:.2f} MWh")

# Bilan : des emissions associées à la chaleur et l'électricité

total_emissions_co2_elec_t = (df["Electricity_Consumption[kW]"] * df["Taux de Co2"]).sum() / 1_000_000  # en tonne
total_emissions_co2_heat_t = (df["Heat_demand[kW]"] * df["cciag_co2"]).sum() / 1_000_000

print(f"Carbon footprint related to electricity consumption : {total_emissions_co2_elec_t:.2f} tonnes")
print(f"Carbon footprint related to heat energy consumption (CCIAG) : {total_emissions_co2_heat_t:.2f} tonnes")

# Bilan : des coûts associés à la chaleur et l'électricité

total_cost_elec = (df["Electricity_Consumption[MW]"] * df["cost_MWh"]).dropna().sum()

total_cost_heat = (df["Heat_demand[MW]"] * df["cciag_cost"]).dropna().sum()

print(f"Total Cost of Electricity : {total_cost_elec:,.2f} €")
print(f"Total Cost of Heat Energy (CCIAG) : {total_cost_heat:,.2f} €")


print("\n--- SUMMARY OF CONSUMPTION, EMISSIONS, AND COSTS AFTER RESCHEDULING ---")

# --- Consommations totales ---

total_elec_aimants_MWh = df["Electricity_Consumption_rearranged[MW]"].dropna().sum()
total_elec_hp_MWh = df["Heat Pump Electricity Consumption (kW)"].dropna().sum() / 1000  # Conversion en MWh
total_heat_demand_MWh = df["Heat_demand[MW]"].dropna().sum()
total_heat_cciag_MWh = df["CCIAG Production (kW)"].dropna().sum() / 1000   # Conversion en MWh

print("\n--- Consumption ---")
print(f"Total electricity consumption (Magnets): {total_elec_aimants_MWh:.2f} MWh")
print(f"Total electricity consumption (Heat pump) : {total_elec_hp_MWh:.2f} MWh")
print(f"Total heat demand (CNRS) : {total_heat_demand_MWh:.2f} MWh")
print(f"Heat production (CCIAG, after replanning) : {total_heat_cciag_MWh:.2f} MWh")

# --- Émissions de CO2 ---
emissions_co2_aimants_t = (df["Electricity_Consumption_rearranged[MW]"] * df["Taux de Co2"]).sum() / 1_000
emissions_co2_hp_t = (df["Heat Pump Electricity Consumption (kW)"] * df["Taux de Co2"]).sum() / 1_000_000
emissions_co2_cciag_t = (df["CCIAG Production (kW)"] * df["cciag_co2"]).sum() / 1_000_000

print("\n---  CO₂ Emissions ---")
print(f"Carbon Emissions related to electricity consumption by Magnets : {emissions_co2_aimants_t:.2f} tonnes")
print(f"Carbon Emissions related to electricity consumption by heat pump : {emissions_co2_hp_t:.2f} tonnes")
print(f"Carbon Emissions related to heating (CCIAG, after replanning) : {emissions_co2_cciag_t:.2f} tonnes")

# --- Coûts associés ---
cost_aimants_elec = (df["Electricity_Consumption_rearranged[MW]"] * df["cost_MWh"]).dropna().sum()
cost_hp_elec = (df["Heat Pump Electricity Consumption (kW)"] / 1000 * df["cost_MWh"]).dropna().sum()
cost_cciag_heat = (df["CCIAG Production (kW)"] / 1000 * df["cciag_cost"]).dropna().sum()

print("\n--- Costs ---")
print(f"Total cost of electricity (Magnets) : {cost_aimants_elec:,.2f} €")
print(f"Total cost of electricity (heatpump) : {cost_hp_elec:,.2f} €")
print(f"Total cost of heating (CCIAG, after replanning) : {cost_cciag_heat:,.2f} €")


--- SUMMARY OF CONSUMPTION, EMISSIONS, AND COSTS of the current LNCMI ---
Total Heat Demand : 2768.12 MWh
Total Electricity Demand : 13773.80 MWh
Carbon footprint related to electricity consumption : 434.19 tonnes
Carbon footprint related to heat energy consumption (CCIAG) : 202.61 tonnes
Total Cost of Electricity : 893,860.77 €
Total Cost of Heat Energy (CCIAG) : 237,841.11 €

--- SUMMARY OF CONSUMPTION, EMISSIONS, AND COSTS AFTER RESCHEDULING ---

--- Consumption ---
Total electricity consumption (Magnets): 13773.80 MWh
Total electricity consumption (Heat pump) : 377.75 MWh
Total heat demand (CNRS) : 2768.12 MWh
Heat production (CCIAG, after replanning) : 1068.25 MWh

---  CO₂ Emissions ---
Carbon Emissions related to electricity consumption by Magnets : 379.58 tonnes
Carbon Emissions related to electricity consumption by heat pump : 13.94 tonnes
Carbon Emissions related to heating (CCIAG, after replanning) : 79.97 tonnes

--- Costs ---
Total cost of electricity (Magnets) : 930,455.

C:\Users\twumduan\AppData\Local\Temp\ipykernel_39928\3996609218.py:6: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

C:\Users\twumduan\AppData\Local\Temp\ipykernel_39928\3996609218.py:16: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [22]:
import pandas as pd

# === EXTRACTION DES DONNÉES ===

# Référence
conso_elec_aimants_ref = df["Electricity_Consumption[MW]"].dropna().sum()
surconso_aimants_ref = 0.0
conso_elec_pac_ref = 0.0
conso_elec_totale_ref = conso_elec_aimants_ref + conso_elec_pac_ref + surconso_aimants_ref
prod_cciag_ref = df["Heat_demand[MW]"].dropna().sum()

co2_elec_aimants_ref = (df["Electricity_Consumption[kW]"] * df["Taux de Co2"]).sum() / 1_000_000
co2_surconso_ref = 0.0
co2_pac_ref = 0.0
co2_cciag_ref = (df["Heat_demand[kW]"] * df["cciag_co2"]).sum() / 1_000_000
co2_total_ref = co2_elec_aimants_ref + co2_surconso_ref + co2_pac_ref + co2_cciag_ref

cost_aimants_ref = (df["Electricity_Consumption[MW]"] * df["cost_MWh"]).dropna().sum()
cost_surconso_ref = 0.0
cost_pac_ref = 0.0
cost_heat_ref = (df["Heat_demand[MW]"] * df["cciag_cost"]).dropna().sum()
cost_total_ref = cost_aimants_ref + cost_surconso_ref + cost_pac_ref + cost_heat_ref

# Valo Hebdo
conso_elec_aimants_valo = df["Electricity_Consumption_rearranged[MW]"].dropna().sum()
conso_elec_pac_valo = df["Heat Pump Electricity Consumption (kW)"].dropna().sum() / 1000
surconso_aimants_valo = 195
conso_elec_totale_valo = conso_elec_aimants_valo + conso_elec_pac_valo + surconso_aimants_valo
prod_cciag_valo = df["CCIAG Production (kW)"].dropna().sum() / 1000

co2_elec_aimants_valo = (df["Electricity_Consumption_rearranged[MW]"] * df["Taux de Co2"]).sum() / 1_000
co2_surconso_valo = surconso_aimants_valo * df["Taux de Co2"].mean() / 1000
co2_pac_valo = (df["Heat Pump Electricity Consumption (kW)"] * df["Taux de Co2"]).sum() / 1_000_000
co2_cciag_valo = (df["CCIAG Production (kW)"] * df["cciag_co2"]).sum() / 1_000_000
co2_total_valo = co2_elec_aimants_valo + co2_pac_valo + co2_cciag_valo + co2_surconso_valo
total_reduction_valo = round(100 * (1 - co2_total_valo / co2_total_ref), 2)

cost_aimants_valo = (df["Electricity_Consumption_rearranged[MW]"] * df["cost_MWh"]).dropna().sum()
cost_surconso_valo = surconso_aimants_valo * df["cost_MWh"].mean()
cost_pac_valo = (df["Heat Pump Electricity Consumption (kW)"] * df["cost_MWh"]).dropna().sum() / 1000
cost_heat_valo = (df["CCIAG Production (kW)"] * df["cciag_cost"]).dropna().sum() / 1000
cost_total_valo = cost_aimants_valo + cost_surconso_valo + cost_pac_valo + cost_heat_valo


from IPython.display import display
import numpy as np

# === CRÉATION DU DATAFRAME RÉSUMÉ ===

data = {
    "Magnet Electricity Consumption [MWh]": [conso_elec_aimants_ref, conso_elec_aimants_valo],
    "Magnet Electricity overconsumption [MWh]": [surconso_aimants_ref, surconso_aimants_valo],
    "Heat pump Electricity consumption [MWh]": [conso_elec_pac_ref, conso_elec_pac_valo],
    "Total Electricity consumption [MWh]": [conso_elec_totale_ref, conso_elec_totale_valo],
    "Production CCIAG [MWh]": [prod_cciag_ref, prod_cciag_valo],
    "CO₂ Elec Magnet [t]": [co2_elec_aimants_ref, co2_elec_aimants_valo],
    "CO₂ Elec Magnet Overconso [t]": [co2_surconso_ref, co2_surconso_valo],
    "CO₂ Elec Heatpump [t]": [co2_pac_ref, co2_pac_valo],
    "CO₂ CCIAG [t]": [co2_cciag_ref, co2_cciag_valo],
    "Total CO₂ [t]": [co2_total_ref, co2_total_valo],
    "Cost Elec Magnets [€]": [cost_aimants_ref, cost_aimants_valo],
    "Cost Magnet Overconso. Elec [€]": [cost_surconso_ref, cost_surconso_valo],
    "Cost Elec Heatpump [€]": [cost_pac_ref, cost_pac_valo],
    "Heating Cost CCIAG [€]": [cost_heat_ref, cost_heat_valo],
    "Total Cost [€]": [cost_total_ref, cost_total_valo],
}

scenarios = ["REF", "VALO-HEBDO"]
df_summary = pd.DataFrame(data, index=scenarios)

# === STYLE D’AFFICHAGE ===

def highlight_columns(style):
    return (
        style
        .set_table_styles([
            {"selector": "th.col0", "props": [("background-color", "#d0ebff")]},   # Conso Elec Aimants
            {"selector": "th.col1", "props": [("background-color", "#d0ebff")]},   # Surconso Elec Aimants
            {"selector": "th.col2", "props": [("background-color", "#d0ebff")]},   # Conso Elec PAC
            {"selector": "th.col3", "props": [("background-color", "#74c0fc"), ("font-weight", "bold")]},  # Conso Elec Totale
            {"selector": "th.col4", "props": [("background-color", "#d3f9d8"), ("font-weight", "bold")]},  # Prod CCIAG
            {"selector": "th.col5", "props": [("background-color", "#ffe066")]},   # CO₂ Elec Aimants
            {"selector": "th.col6", "props": [("background-color", "#ffe066")]},   # CO₂ Surconso Elec
            {"selector": "th.col7", "props": [("background-color", "#ffe066")]},   # CO₂ Elec PAC
            {"selector": "th.col8", "props": [("background-color", "#ffe066")]},   # CO₂ CCIAG
            {"selector": "th.col9", "props": [("background-color", "#fab005"), ("font-weight", "bold")]},  # CO₂ Total
            {"selector": "th.col10", "props": [("background-color", "#dee2e6"), ("font-weight", "bold")]}, # Coût Elec Aimants
            {"selector": "th.col11", "props": [("background-color", "#dee2e6"), ("font-weight", "bold")]}, # Coût Surconso
            {"selector": "th.col12", "props": [("background-color", "#dee2e6"), ("font-weight", "bold")]}, # Coût PAC
            {"selector": "th.col13", "props": [("background-color", "#dee2e6"), ("font-weight", "bold")]}, # Coût Chaleur
            {"selector": "th.col14", "props": [("background-color", "#ced4da"), ("font-weight", "bold")]}  # Coût Total
        ])
        .set_properties(subset=["Total Electricity consumption [MWh]",
                                "Production CCIAG [MWh]",
                                "Total CO₂ [t]",
                                "Total Cost [€]"], **{"font-weight": "bold"})
        .format("{:,.2f}")
    )

# AFFICHAGE FINAL
styled_full = highlight_columns(df_summary.style)
display(styled_full)


# -------------------------------------------------------------------------------------------------------------------------

ref_hebdo_data = {
    "Elec [MWh]": 13773.8,
    "Heating [MWh]": 2768.12,
    "CO₂ Elec [t]": 349.53,
    "CO₂ heating [t]": 202.61,
    "Total CO₂ [t]": 552.14,
    "Total cost [€]": 1874008,
}





data_simplified = {
    "Elec [MWh]": [conso_elec_totale_ref, conso_elec_totale_valo],
    "Heating [MWh]": [prod_cciag_ref, prod_cciag_valo],
    "CO₂ Elec [t]": [co2_elec_aimants_ref + co2_pac_ref + co2_surconso_ref,
                               co2_elec_aimants_valo + co2_pac_valo + co2_surconso_valo],
    "CO₂ heating [t]": [co2_cciag_ref, co2_cciag_valo],
    "Total CO₂ [t]": [co2_total_ref, co2_total_valo],
     "Total cost [€]": [cost_total_ref, cost_total_valo]
}

df_summary_simple = pd.DataFrame(data_simplified, index=["REF", "VALO-HEBDO"])
df_summary_simple.loc["REF-HEBDO"] = ref_hebdo_data
df_summary_simple = df_summary_simple.loc[["REF", "REF-HEBDO", "VALO-HEBDO"]]

# === MISE EN FORME DU TABLEAU SIMPLIFIÉ ===

def style_simple_table(style):
    return (
        style
        .set_table_styles([
            {"selector": "th.col0", "props": [("background-color", "#74c0fc"), ("font-weight", "bold")]},  # Conso Elec
            {"selector": "th.col1", "props": [("background-color", "#d3f9d8"), ("font-weight", "bold")]},  # Conso Thermique
            {"selector": "th.col2", "props": [("background-color", "#ffd43b")]},  # CO₂ Elec
            {"selector": "th.col3", "props": [("background-color", "#ffd43b")]},  # CO₂ CCIAG
            {"selector": "th.col4", "props": [("background-color", "#fab005"), ("font-weight", "bold")]}   # CO₂ Total
        ])
        .set_properties(subset=[ "Elec [MWh]",
                                 "Heating [MWh]",
                                "Total CO₂ [t]"], **{"font-weight": "bold"})
        .format("{:,.0f}", na_rep="")
    )

# AFFICHAGE FINAL
styled_simple = style_simple_table(df_summary_simple.style)
display(styled_simple)

,Magnet Electricity Consumption [MWh],Magnet Electricity overconsumption [MWh],Heat pump Electricity consumption [MWh],Total Electricity consumption [MWh],Production CCIAG [MWh],CO₂ Elec Magnet [t],CO₂ Elec Magnet Overconso [t],CO₂ Elec Heatpump [t],CO₂ CCIAG [t],Total CO₂ [t],Cost Elec Magnets [€],Cost Magnet Overconso. Elec [€],Cost Elec Heatpump [€],Heating Cost CCIAG [€],Total Cost [€]
REF,"13,773.80",0.00,0.00,"13,773.80","2,768.12",434.19,0.00,0.00,202.61,636.79,"893,860.77",0.00,0.00,"237,841.11","1,131,701.88"
VALO-HEBDO,"13,773.80",195.00,377.75,"14,346.55","1,068.25",379.58,6.74,13.94,79.97,480.24,"930,455.30","13,691.07","33,347.86","93,482.54","1,070,976.78"


,Elec [MWh],Heating [MWh],CO₂ Elec [t],CO₂ heating [t],Total CO₂ [t],Total cost [€]
REF,"13,774","2,768",434,203,637,"1,131,702"
REF-HEBDO,"13,774","2,768",350,203,552,"1,874,008"
VALO-HEBDO,"14,347","1,068",400,80,480,"1,070,977"
